
### 02_data_analysis.ipynb
```markdown
# 02_data_analysis.ipynb

## Notebook Purpose
This notebook is designed to perform exploratory data analysis (EDA) on the preprocessed cryptocurrency data. It will generate visualizations to help understand the data, identify trends, and detect any patterns.

## Instructions
1. **Import Necessary Libraries**:
   - Import `pandas` for data manipulation.
   - Import `matplotlib` and `seaborn` for data visualization.

2. **Load Preprocessed Data**:
   - Load the preprocessed CSV file created in the previous notebook.

3. **Plot Closing Prices**:
   - Generate a plot showing the closing prices over time to observe overall trends.

4. **Plot Technical Indicators**:
   - Generate plots for the calculated technical indicators (e.g., SMA, EMA) to visualize their behavior over time.

5. **Correlation Analysis**:
   - Calculate and visualize the correlation matrix to understand relationships between different features.

## Example Code
```python
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load preprocessed data
data_path = 'data/historical_data/btc_usd_preprocessed.csv'  # Update this path based on the selected cryptocurrency
data = pd.read_csv(data_path, parse_dates=['Date'], index_col='Date')

# Plot closing price over time
plt.figure(figsize=(14, 7))
plt.plot(data['Close'])
plt.title('Closing Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.show()

# Plot technical indicators
plt.figure(figsize=(14, 7))
plt.plot(data['Close'], label='Close')
plt.plot(data['SMA_20'], label='SMA 20')
plt.plot(data['SMA_50'], label='SMA 50')
plt.title('Technical Indicators')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

# Display correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()
